In [18]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [19]:
# Leer el archivo CSV
df_incendios = pd.read_csv(r'G:\Mi unidad\OSPA\01. Tematicas\04. Incendios\01. Productos\postmodelo_icv\temporales\probabilidad_icv.csv', sep=';', encoding='utf-8')
df_deslizamientos = pd.read_csv(r'G:\Mi unidad\OSPA\01. Tematicas\03. Deslizamientos\01. Productos\postmodelo_idd\temporales\probabilidad_idd.csv', sep=';', encoding='utf-8')

In [20]:
# Crear una conexión a la base de datos
usuario = 'postgres'
contraseña = 'Septiembre0672'
host = 'localhost'
puerto = '5432'
db = 'alertas'
url_conexion = f'postgresql://{usuario}:{contraseña}@{host}:{puerto}/{db}'

In [21]:
def alertas_incendios(url, df_incendios):
    try:
        engine = create_engine(url_conexion)
        return df_incendios.to_sql('incendios', con=engine, if_exists='replace', index=False)
    except:
        engine = create_engine(url_conexion)
        # Eliminar los registros existentes de la tabla 'incendios'
        with engine.connect() as conn:
            conn.execute("DELETE FROM incendios")
        return df_incendios.to_sql('incendios', con=engine, if_exists='append', index=False)

In [22]:
alertas_incendios(url_conexion, df_incendios)

In [23]:
#engine = create_engine(url_conexion)

In [24]:
# Eliminar los registros existentes de la tabla 'incendios'
# with engine.connect() as conn:
#     conn.execute("DELETE FROM incendios")

In [25]:
# df_incendios.to_sql('incendios', con=engine, if_exists='append', index=False)

In [26]:
def alertas_deslizamientos(url, df_deslizamientos):
    try:
        engine = create_engine(url_conexion)
        return df_deslizamientos.to_sql('deslizamientos', con=engine, if_exists='replace', index=False)
    except:
        engine = create_engine(url_conexion)
        # Eliminar los registros existentes de la tabla 'incendios'
        with engine.connect() as conn:
            conn.execute("DELETE FROM deslizamientos")
        return df_deslizamientos.to_sql('deslizamientos', con=engine, if_exists='append', index=False)

In [27]:
alertas_deslizamientos(url_conexion, df_deslizamientos)

In [13]:
# engine_2 = create_engine(url_conexion)

In [14]:
# # Eliminar los registros existentes de la tabla 'incendios'
# with engine_2.connect() as conn:
#     conn.execute("DELETE FROM deslizamientos")

In [15]:
# df_deslizamientos.to_sql('deslizamientos', con=engine, if_exists='append', index=False)